- __01)One Hot Encoding__
- __02)missing data__

In [5]:
import pandas as pd
import numpy as np

In [6]:
data_raw = pd.read_csv("G:\\Data_analyst\\05)data prepration\\heart.csv")
data = data_raw.copy()

In [7]:
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


#### Replacing 0 numbers which are our null numbers in RestingBP and Cholesterol.

In [8]:
data.replace({"RestingBP" : 0, "Cholesterol" : 0 }, np.nan,inplace=True)

In [9]:
data.isna().sum()

Age                 0
Sex                 0
ChestPainType       0
RestingBP           1
Cholesterol       172
FastingBS           0
RestingECG          0
MaxHR               0
ExerciseAngina      0
Oldpeak             0
ST_Slope            0
HeartDisease        0
dtype: int64

## 01)One Hot Encoding

#### Separating the data into number part and object part in order to do One_Hot_Encoding.

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       917 non-null    float64
 4   Cholesterol     746 non-null    float64
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 86.2+ KB


In [11]:
data_number = data.select_dtypes(["float64","int64"]).iloc[:,:-1]
data_object = data.select_dtypes("object")
data_label  = data.iloc[:,-1].values

##### labeling objects in rows:

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
label_encoder = LabelEncoder()

In [14]:
data_object = data_object.apply(label_encoder.fit_transform)

In [15]:
data_object

,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope
0,1,1,1,0,2
1,0,2,1,0,1
2,1,1,2,0,2
3,0,0,1,1,1
4,1,2,1,0,2
...,...,...,...,...,...
913,1,3,1,0,1
914,1,0,1,0,1
915,1,0,1,1,1
916,0,1,0,0,1


In [16]:
data_object.nunique()

Sex               2
ChestPainType     4
RestingECG        3
ExerciseAngina    2
ST_Slope          3
dtype: int64

__Since "Sex" and "ExerciseAngina" have only 2 values (0 and 1) we do not implement on hot encoding on them, in order not to meke more columns.__


#### separatin sex and ExerciseAngina columns:

In [17]:
data_object_binary = data_object[["Sex" , "ExerciseAngina"]]

In [18]:
data_object_ohe = data_object[["ChestPainType" , "RestingECG" , "ST_Slope"]]

#### One_Hot_Encoding

In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
ohe = OneHotEncoder()

In [21]:
data_object_ohe = ohe.fit_transform(data_object_ohe[["ChestPainType" , "RestingECG" , "ST_Slope"]].values).toarray()

In [22]:
print(pd.DataFrame(data_object_ohe))

       0    1    2    3    4    5    6    7    8    9
0    0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
1    0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
2    0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0
3    1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
4    0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
913  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0
914  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
915  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
916  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0
917  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0

[918 rows x 10 columns]


#### Conacat all separated data

In [23]:
data_number.shape

(918, 6)

In [20]:
data_final = np.concatenate([data_number.values , data_object_binary.values , data_object_ohe], axis=1)

In [21]:
print(pd.DataFrame(data_final))

       0      1      2    3      4    5    6    7    8    9    10   11   12  \
0    40.0  140.0  289.0  0.0  172.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0   
1    49.0  160.0  180.0  0.0  156.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
2    37.0  130.0  283.0  0.0   98.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0   
3    48.0  138.0  214.0  0.0  108.0  1.5  0.0  1.0  1.0  0.0  0.0  0.0  0.0   
4    54.0  150.0  195.0  0.0  122.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
..    ...    ...    ...  ...    ...  ...  ...  ...  ...  ...  ...  ...  ...   
913  45.0  110.0  264.0  0.0  132.0  1.2  1.0  0.0  0.0  0.0  0.0  1.0  0.0   
914  68.0  144.0  193.0  1.0  141.0  3.4  1.0  0.0  1.0  0.0  0.0  0.0  0.0   
915  57.0  130.0  131.0  0.0  115.0  1.2  1.0  1.0  1.0  0.0  0.0  0.0  0.0   
916  57.0  130.0  236.0  0.0  174.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
917  38.0  138.0  175.0  0.0  173.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0   

      13   14   15   16   17  
0    1.0  0.0  0.0  

##  02)missing data

__KNNIMputer__

In [22]:
from sklearn.impute import KNNImputer

In [23]:
knn = KNNImputer()

In [24]:
data_nan = knn.fit_transform(data_final)

In [25]:
pd.DataFrame(data_nan)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,40.0,140.0,289.0,0.0,172.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,49.0,160.0,180.0,0.0,156.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,37.0,130.0,283.0,0.0,98.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,48.0,138.0,214.0,0.0,108.0,1.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,54.0,150.0,195.0,0.0,122.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45.0,110.0,264.0,0.0,132.0,1.2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
914,68.0,144.0,193.0,1.0,141.0,3.4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
915,57.0,130.0,131.0,0.0,115.0,1.2,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
916,57.0,130.0,236.0,0.0,174.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [27]:
pd.DataFrame(data_nan).isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
dtype: int64